In [1]:
DF_PATH       = "../data/processed/2_cleaned_data.pkl"

ROLE_COLS      = ['DevType']
TECH_COLS      = ['LanguageWorkedWith',
                  'DatabaseWorkedWith',
                  'WebframeWorkedWith',
                  'MiscTechWorkedWith']

MLFLOW_TRACKING_URI = '../models/mlruns'
MLFLOW_EXPERIMENT_NAME = "skills_jobs_stackoverflow"

LOG_PATH = "../models/temp/"
LOG_DATA_PKL    =  "data.pkl"
LOG_MODEL_PKL   =  "model.pkl"
LOG_METRICS_PKL =  "metrics.pkl"

In [2]:
# Load packages
import pandas as pd 
import numpy as np
import logging
import pickle
import random
import plotly 
import os
from pathlib import Path

import mlflow
from mlflow.tracking import MlflowClient

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.pipeline import make_pipeline, FeatureUnion
from sklearn.feature_selection import VarianceThreshold
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn import metrics
from sklearn.metrics import auc, accuracy_score, confusion_matrix, f1_score, precision_score, recall_score

from sklearn.decomposition import PCA, KernelPCA

from sklearn import tree
from sklearn.linear_model import LogisticRegression

from matplotlib import pyplot as plt

## Functions

In [4]:
def calculate_quality(ground_truth, predictions, metric_function, sort_values = False):
    """ .)So this function answers: How good is my model for each role separately?
        .)It is doing: Per-label evaluation"""
    quality_scores = {}
    for col in predictions.columns:
        role_pred = predictions[col].copy()
        role_truth = ground_truth[col].copy()
        quality_scores[col] = round(metric_function(role_truth, role_pred)*100,2)
    quality_scores = pd.Series(quality_scores.vlaues(), index = quality_scores.keys())
    if sort_values:
        quality_scores = quality_scores.sort_values()
    return quality_scores

## Initialize

### Create Directories

In [5]:
Path(MLFLOW_TRACKING_URI).mkdir(parents=True, exist_ok = True)
Path(LOG_PATH).mkdir(parents=True, exist_ok = True)

### read data

In [7]:
df = pd.read_pickle(DF_PATH)
df

DevType                           \
      Academic researcher Data or business analyst   
0                       0                        0   
1                       0                        0   
5                       0                        0   
7                       0                        0   
8                       0                        0   
...                   ...                      ...   
64416                   0                        1   
64422                   0                        0   
64428                   0                        0   
64447                   0                        0   
64453                   0                        1   

                                                                            \
      Data scientist or machine learning specialist Database administrator   
0                                                 0                      0   
1                                                 0                      0   
5                                                 0                      0   
7                                                 0                      0   
8                                                 0                      0   
...                                             ...                    ...   
64416                                             0                      0   
64422                                             0                      0   
64428                                             0                      0   
64447                                             0                      0   
64453                                             0                      0   

                                                                   \
      DevOps specialist Developer, QA or test Developer, back-end   
0                     0                     0                   0   
1                     0                     0                   0   
5                     0                     0                   0   
7                     0                     0                   1   
8                     0                     0                   0   
...                 ...                   ...                 ...   
64416                 0                     0                   0   
64422                 0                     0                   1   
64428                 0                     0                   0   
64447                 0                     0                   1   
64453                 0                     0                   0   

                                                     \
      Developer, desktop or enterprise applications   
0                                                 1   
1                                                 0   
5                                                 0   
7                                                 1   
8                                                 0   
...                                             ...   
64416                                             0   
64422                                             0   
64428                                             0   
64447                                             0   
64453                                             0   

                                                                        ...  \
      Developer, embedded applications or devices Developer, front-end  ...   
0                                               0                    0  ...   
1                                               0                    0  ...   
5                                               0                    1  ...   
7                                               0                    0  ...   
8                                               0                    0  ...   
...                                           ...                  ...  ...   
64416                                           0     

## Balance classes

In [8]:
# Check the total samples of roles
roles_df = df["DevType"].copy()
roles_df.sum()

Academic researcher                              1039
Data or business analyst                         1059
Data scientist or machine learning specialist    1275
Database administrator                            745
DevOps specialist                                1212
Developer, QA or test                             791
Developer, back-end                              9144
Developer, desktop or enterprise applications    2985
Developer, embedded applications or devices      1193
Developer, front-end                             5177
Developer, full-stack                            8718
Developer, game or graphics                       608
Developer, mobile                                2573
Engineer, data                                    916
Scientist                                         590
System administrator                              880
dtype: int64

In [10]:
# Resample roles
samples_per_class = 700
resampled_roles = []
for role in roles_df:
    sub_df = roles_df.loc[roles_df[role] ==1].copy()
    if len(sub_df) <samples_per_class:
        # UpScale
        sub_df = sub_df.sample(samples_per_class, replace = True, random_state = 0)
    else:
        sub_df = sub_df.sample(samples_per_class, random_state = 0)
    resampled_roles.append(sub_df)

In [16]:
roles_df = pd.concat(resampled_roles)
roles_df.sum()

Academic researcher                              1458
Data or business analyst                         1257
Data scientist or machine learning specialist    1528
Database administrator                           1127
DevOps specialist                                1222
Developer, QA or test                             967
Developer, back-end                              3892
Developer, desktop or enterprise applications    1831
Developer, embedded applications or devices      1111
Developer, front-end                             1782
Developer, full-stack                            2985
Developer, game or graphics                       922
Developer, mobile                                1357
Engineer, data                                   1159
Scientist                                        1149
System administrator                             1194
dtype: int64

In [17]:
df = df.loc[roles_df.index].copy()

In [19]:
X_train,X_test,y_train,y_test = train_test_split(df.drop("DevType", axis=1), df["DevType"], random_state = 0)

C:\Users\user\AppData\Local\Temp\ipykernel_11620\2756033559.py:1: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  X_train,X_test,y_train,y_test = train_test_split(df.drop("DevType", axis=1), df["DevType"], random_state = 0)


## Train models

### Initialize MLflow

In [20]:
#Initialize Client and Experiment
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
client = MlflowClient()
mlflow.get_experiment_by_name(MLFLOW_EXPERIMENT_NAME)
exp = client.get_experiment_by_name(MLFLOW_EXPERIMENT_NAME)

C:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\mlflow\tracking\_tracking_service\utils.py:177: FutureWarning: The filesystem tracking backend (e.g., './mlruns') will be deprecated in February 2026. Consider transitioning to a database backend (e.g., 'sqlite:///mlflow.db') to take advantage of the latest MLflow features. See https://github.com/mlflow/mlflow/issues/18534 for more details and migration guidance.
  return FileStore(store_uri, store_uri)


### 1. Logistic Regression

In [23]:
clf = make_pipeline(StandardScaler(),
                   MultiOutputClassifier(LogisticRegression()))
clf.fit(X_train.values, y_train.values)
predictions = pd.DataFrame(clf.predict(X_train.values), columns = y_train.columns)